In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib as plt

In [ ]:
%matplotlib inline

In [ ]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df = df.fillna('NaN')
df_test = df_test.fillna('NaN')

In [ ]:
dates_countries = df.groupby(['Date', 'Country_Region'], as_index=False)\
                    .agg({'ConfirmedCases': 'sum', 'Fatalities': 'sum'})

dates_countries['Date'] = pd.to_datetime(dates_countries['Date'])
training_mask = np.logical_and(pd.to_datetime('2020-01-19') < dates_countries['Date'],
                               dates_countries['Date'] < pd.to_datetime('2020-03-19'))
data = dates_countries[training_mask].copy()
data.loc[:, ('ConfirmedCases', 'Fatalities')] = data.loc[:, ['ConfirmedCases', 'Fatalities']] \
                                                .apply(lambda x: np.log1p(x))
data.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
data.loc[:, ('ConfirmedCases', 'Fatalities')] = data.loc[:, ['ConfirmedCases', 'Fatalities']] \
                                                .apply(lambda x: np.log1p(x))
data.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
def lagging(df, lags=10):
    dfs = []
    for i in range(1, lags):
        lag_df = df.shift(i, fill_value=0)
        lag_df = lag_df.rename(lambda x: x + f'_{i}', axis=1)
        dfs.append(lag_df)
    
    return pd.concat(dfs, axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
data['Day_num'] = le.fit_transform(data.Date)
data['Day'] = data['Date'].dt.day
data['Month'] = data['Date'].dt.month

In [ ]:
country_dfs = []
for country in ['Italy']:#dates_countries['Country_Region'].unique():
    country_mask = data['Country_Region'] == country
    country_df = data.loc[country_mask]
    lags_df = lagging(country_df[['ConfirmedCases', 'Fatalities']])
    country_df = pd.concat([country_df, lags_df], axis=1).dropna()

In [ ]:
country_df